### Two-Tower Model

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Add, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [5]:
movies_df = pd.read_csv('/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/movie_info.csv')
ratings_df = pd.read_csv('/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/ratings_all_development_set.csv')
ratings_df = ratings_df.rename(columns={'user_id': 'userId', 'item_id': 'movieId'})
movies_df = movies_df.rename(columns={'item_id': 'movieId'})
leaderboard_data = pd.read_csv('/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/ratings_masked_leaderboard_set.csv') 

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# Ensure required columns exist
assert 'userId' in ratings_df.columns and 'movieId' in ratings_df.columns and 'rating' in ratings_df.columns

# Encode user and movie IDs
user_encoder = LabelEncoder()
ratings_df['user_id_encoded'] = user_encoder.fit_transform(ratings_df['userId'])

movie_encoder = LabelEncoder()
ratings_df['movie_id_encoded'] = movie_encoder.fit_transform(ratings_df['movieId'])

# Copy the dataset and split into train and test sets
df = ratings_df.copy()
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Convert columns to NumPy arrays
train_user_ids = np.array(train['user_id_encoded'].values)
train_movie_ids = np.array(train['movie_id_encoded'].values)
train_ratings = np.array(train['rating'].values)

test_user_ids = np.array(test['user_id_encoded'].values)
test_movie_ids = np.array(test['movie_id_encoded'].values)
test_ratings = np.array(test['rating'].values)

# Define the number of unique users, movies, and embedding dimensions
num_users = df['user_id_encoded'].nunique()
num_movies = df['movie_id_encoded'].nunique()
embedding_dim = 50

# Define the model
# User input and embedding
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=num_users + 1, output_dim=embedding_dim, name='user_embedding')(user_input)
user_embedding = Flatten()(user_embedding)

# Movie input and embedding
movie_input = Input(shape=(1,), name='movie_input')
movie_embedding = Embedding(input_dim=num_movies + 1, output_dim=embedding_dim, name='movie_embedding')(movie_input)
movie_embedding = Flatten()(movie_embedding)

# Dot product of embeddings and output layer
dot_product = Dot(axes=1)([user_embedding, movie_embedding])
output = Dense(1, activation='linear')(dot_product)

# Compile the model
model2 = Model(inputs=[user_input, movie_input], outputs=output)
model2.compile(optimizer='adam', loss='mae')

# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model2.fit(
    [train_user_ids, train_movie_ids], train_ratings,
    epochs=6,
    batch_size=64,
    validation_data=([test_user_ids, test_movie_ids], test_ratings),
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss = model2.evaluate([test_user_ids, test_movie_ids], test_ratings)
print(f"Test loss (MAE): {test_loss}")

# Predict ratings for the test set
test_predictions = model2.predict([test_user_ids, test_movie_ids])

# Create a DataFrame with predictions
test_results = pd.DataFrame({
    'original_user_id': test['userId'],
    'original_movie_id': test['movieId'],
    'actual_rating': test_ratings,
    'predicted_rating': test_predictions.flatten()
})
test_results['rating_difference'] = abs(test_results['actual_rating'] - test_results['predicted_rating'])

# Filter results for a specific user and sort by rating difference
filter_user_id = 772  # Replace with your desired user ID
filtered_results = test_results[test_results['original_user_id'] == filter_user_id]
filtered_results_sorted = filtered_results.sort_values(by='rating_difference')

print(f"Predictions for user {filter_user_id}, ordered by closest rating difference:")
print(filtered_results_sorted)

# Load leaderboard data
leaderboard_data = pd.read_csv('/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/ratings_masked_leaderboard_set.csv')  # Replace with your leaderboard file path
assert 'user_id' in leaderboard_data.columns and 'item_id' in leaderboard_data.columns

# Handle unseen user and movie IDs using mapping
user_mapping = dict(zip(user_encoder.classes_, user_encoder.transform(user_encoder.classes_)))
movie_mapping = dict(zip(movie_encoder.classes_, movie_encoder.transform(movie_encoder.classes_)))

# Map user and movie IDs, assigning unseen IDs to placeholder index
leaderboard_data['user_id_encoded'] = leaderboard_data['user_id'].map(
    lambda x: user_mapping.get(x, num_users)
).astype(int)

leaderboard_data['item_id_encoded'] = leaderboard_data['item_id'].map(
    lambda x: movie_mapping.get(x, num_movies)
).astype(int)

# Predict leaderboard ratings
user_ids = leaderboard_data['user_id_encoded'].values
item_ids = leaderboard_data['item_id_encoded'].values
predictions = model2.predict([user_ids, item_ids]).flatten()

# Save leaderboard predictions to a file
with open("predicted_ratings_leaderboard3.txt", "w") as f:
    for pred in predictions:
        f.write(f"{pred:.4f}\n")


Epoch 1/6
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 2.9531 - val_loss: 0.8614
Epoch 2/6
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.8016 - val_loss: 0.7686
Epoch 3/6
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7368 - val_loss: 0.7539
Epoch 4/6
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7089 - val_loss: 0.7452
Epoch 5/6
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6837 - val_loss: 0.7408
Epoch 6/6
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6632 - val_loss: 0.7368
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - loss: 0.7405
Test loss (MAE): 0.736820638179779
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step
Predictions for user 772, ordered by closest rating difference:
       original_user_id  original_movie_id  actual_rating  predicted_rating  \
525                 772                175              4          3.934911   
67026               772                893              2          2.241948   
44904               772                 6

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# File paths (replace with actual paths to your files)
ratings_file = '/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/ratings_all_development_set.csv'
user_info_file = '/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/user_info.csv'
movies_file = '/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/movie_info.csv'
leaderboard_file = '/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/ratings_masked_leaderboard_set.csv'

# Read the data from the files
ratings_df = pd.read_csv(ratings_file)
user_info_df = pd.read_csv(user_info_file)
movies_df = pd.read_csv(movies_file)

# Rename the columns in ratings_df to match the expected names
ratings_df.rename(columns={'user_id': 'userId', 'item_id': 'movieId'}, inplace=True)

# Ensure required columns exist
assert 'userId' in ratings_df.columns and 'movieId' in ratings_df.columns and 'rating' in ratings_df.columns
assert 'item_id' in movies_df.columns and 'orig_item_id' in movies_df.columns
assert 'user_id' in user_info_df.columns

# Encode user and movie IDs
user_encoder = LabelEncoder()
ratings_df['user_id_encoded'] = user_encoder.fit_transform(ratings_df['userId'])

movie_encoder = LabelEncoder()
ratings_df['movie_id_encoded'] = movie_encoder.fit_transform(ratings_df['movieId'])

# Merge with additional user and movie data
ratings_df = ratings_df.merge(user_info_df[['user_id', 'age', 'is_male']], left_on='userId', right_on='user_id', how='left')
ratings_df = ratings_df.merge(movies_df[['item_id', 'release_year']], left_on='movieId', right_on='item_id', how='left')

# Convert categorical features into numeric
ratings_df['age'] = ratings_df['age'].astype(int)
ratings_df['is_male'] = ratings_df['is_male'].astype(int)
ratings_df['release_year'] = ratings_df['release_year'].astype(int)

# Copy the dataset and split into train and test sets
df = ratings_df.copy()
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Convert columns to NumPy arrays
train_user_ids = np.array(train['user_id_encoded'].values)
train_movie_ids = np.array(train['movie_id_encoded'].values)
train_ratings = np.array(train['rating'].values)

test_user_ids = np.array(test['user_id_encoded'].values)
test_movie_ids = np.array(test['movie_id_encoded'].values)
test_ratings = np.array(test['rating'].values)

# Define the number of unique users, movies, and embedding dimensions
num_users = df['user_id_encoded'].nunique()
num_movies = df['movie_id_encoded'].nunique()
embedding_dim = 50

# Define the model
# User input and embedding
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=num_users + 1, output_dim=embedding_dim, name='user_embedding')(user_input)
user_embedding = Flatten()(user_embedding)

# Movie input and embedding
movie_input = Input(shape=(1,), name='movie_input')
movie_embedding = Embedding(input_dim=num_movies + 1, output_dim=embedding_dim, name='movie_embedding')(movie_input)
movie_embedding = Flatten()(movie_embedding)

# User features (age, is_male)
user_age_input = Input(shape=(1,), name='user_age')
user_is_male_input = Input(shape=(1,), name='user_is_male')
user_features = Concatenate()([user_age_input, user_is_male_input])

# Movie features (release_year)
movie_release_year_input = Input(shape=(1,), name='movie_release_year')
movie_features = movie_release_year_input

# Concatenate all features (user, movie, user features, movie features)
concatenated = Concatenate()([user_embedding, movie_embedding, user_features, movie_features])

# Fully connected layers
x = Dense(256, activation='relu')(concatenated)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# Output layer
output = Dense(1, activation='linear')(x)

# Compile the model
model2 = Model(inputs=[user_input, movie_input, user_age_input, user_is_male_input, movie_release_year_input], outputs=output)
model2.compile(optimizer=Adam(learning_rate=0.001), loss='mae')

# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model2.fit(
    [train_user_ids, train_movie_ids, train['age'], train['is_male'], train['release_year']],
    train_ratings,
    epochs=10,
    batch_size=64,
    validation_data=([test_user_ids, test_movie_ids, test['age'], test['is_male'], test['release_year']], test_ratings),
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss = model2.evaluate([test_user_ids, test_movie_ids, test['age'], test['is_male'], test['release_year']], test_ratings)
print(f"Test loss (MAE): {test_loss}")

# Predict ratings for the test set
test_predictions = model2.predict([test_user_ids, test_movie_ids, test['age'], test['is_male'], test['release_year']])

# Create a DataFrame with predictions
test_results = pd.DataFrame({
    'original_user_id': test['userId'],
    'original_movie_id': test['movieId'],
    'actual_rating': test_ratings,
    'predicted_rating': test_predictions.flatten()
})
test_results['rating_difference'] = abs(test_results['actual_rating'] - test_results['predicted_rating'])

# Filter results for a specific user and sort by rating difference
filter_user_id = 772  # Replace with your desired user ID
filtered_results = test_results[test_results['original_user_id'] == filter_user_id]
filtered_results_sorted = filtered_results.sort_values(by='rating_difference')

print(f"Predictions for user {filter_user_id}, ordered by closest rating difference:")
print(filtered_results_sorted)

# Predict leaderboard ratings
leaderboard_data = pd.read_csv(leaderboard_file)  # Replace with your leaderboard file path
leaderboard_data['user_id_encoded'] = leaderboard_data['user_id'].map(
    lambda x: user_mapping.get(x, num_users)
).astype(int)
leaderboard_data['item_id_encoded'] = leaderboard_data['item_id'].map(
    lambda x: movie_mapping.get(x, num_movies)
).astype(int)

# Predict leaderboard ratings
user_ids = leaderboard_data['user_id_encoded'].values
item_ids = leaderboard_data['item_id_encoded'].values
leaderboard_predictions = model2.predict([user_ids, item_ids, leaderboard_data['age'], leaderboard_data['is_male'], leaderboard_data['release_year']]).flatten()

# Save leaderboard predictions to a file
with open("predicted_ratings_leaderboard3.txt", "w") as f:
    for pred in leaderboard_predictions:
        f.write(f"{pred:.4f}\n")


Epoch 1/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 6.5868 - val_loss: 0.8163
Epoch 2/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.9569 - val_loss: 0.9154
Epoch 3/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8116 - val_loss: 0.7778
Epoch 4/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.7906 - val_loss: 0.7856
Epoch 5/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7536 - val_loss: 0.7674
Epoch 6/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7503 - val_loss: 0.7680
Epoch 7/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.7334 - val_loss: 0.7925
Epoch 8/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.7325 - val_loss: 0.7422
Epoch 9/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.7375 - val_loss: 0.7485
Epoch 10/10
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.7309 - val_loss: 0.7440
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - loss: 0.7430
Test loss (MAE): 0.7422206997871399
563/563 ━━━━━━━━━━━━━━━━━

NameError: name 'user_mapping' is not defined